The related data and code are also been found on https://github.com/Xuanstar42/BERN02_Exercise3.git

# 1. Import Package

In [30]:
try:
    from scipy.signal import gaussian  
except Exception:
    from scipy.signal.windows import gaussian as _gaussian  
    import scipy.signal as _sig
    _sig.gaussian = _gaussian 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import bambi as bmb
import arviz as az
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# 2. Data Wrangling

In [11]:
# read in data
data_file = "towelData.csv"
data = pd.read_csv(data_file, sep=';', encoding='latin1')
count = data.iloc[:, -1] # get the last column with numbers or yes/no

# count has the number of yes and no for control and social norm groups
control_yes = count[::4].to_numpy() # every 4th starting from 0 - control group + yes
control_no = count[2::4].to_numpy() # every 4th starting from 2 - control group + no
control_total = np.array([y + n for y, n in zip(control_yes, control_no)])

social_yes = count[1::4].to_numpy() # every 4th starting from 1 - social norm group + yes
social_no = count[3::4].to_numpy() # every 4th starting from 3 - social norm group + no
social_total = np.array([y + n for y, n in zip(social_yes, social_no)])

study = np.arange(1,len(control_yes)+1) # 7 diferent studies

control_data = pd.DataFrame({"reuse": control_yes, "total": control_total, "group": "control", "study": study})
social_data = pd.DataFrame({ "reuse": social_yes, "total": social_total, "group": "social", "study": study})

combined_data = pd.concat([control_data, social_data], ignore_index=True)

# Convert data types - important for bambi that they are correctly set 
# can give errors otherwise
combined_data['reuse'] = combined_data['reuse'].astype(int)
combined_data['total'] = combined_data['total'].astype(int)
combined_data['group'] = combined_data['group'].astype('category')
combined_data['study'] = combined_data['study'].astype('category')
print(combined_data.head(25))

    reuse  total    group study
0      74    211  control     1
1     103    277  control     2
2      77    135  control     3
3      82    187  control     4
4      21     25  control     5
5     123    147  control     6
6      28     30  control     7
7      98    222   social     1
8     587   1318   social     2
9     406    655   social     3
10    278    555   social     4
11     21     24   social     5
12    472    576   social     6
13    101    132   social     7


# 3. Apply bmb Model

In [24]:
model_hierarchical = bmb.Model("p(reuse, total) ~ 1 + group + (1|study)", combined_data, family="binomial")
model_hierarchical



       Formula: p(reuse, total) ~ 1 + group + (1|study)
        Family: binomial
          Link: p = logit
  Observations: 14
        Priors: 
    target = p
        Common-level effects
            Intercept ~ Normal(mu: 0.0, sigma: 3.5355)
            group ~ Normal(mu: 0.0, sigma: 5.0)
        
        Group-level effects
            1|study ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 3.5355))

In [26]:
idata_hierarchical = model_hierarchical.fit(random_seed=42)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, group, 1|study_sigma, 1|study_offset]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 15 seconds.
There were 22 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


It was found that there are too many divergences. Therefore, we need to modify the prior.

# 4. Prior Modification

In [27]:
idata_prior = model_hierarchical.prior_predictive()
prior = az.extract_dataset(idata_prior, group="prior_predictive")["p(reuse, total)"]

Sampling: [1|study_offset, 1|study_sigma, Intercept, group, p(reuse, total)]
/var/folders/4s/2wy10by568x979b5g7g66m7c0000gn/T/ipykernel_37961/632727353.py:2: FutureWarning: extract_dataset has been deprecated, please use extract
  prior = az.extract_dataset(idata_prior, group="prior_predictive")["p(reuse, total)"]


In [41]:
priors = {
    "Intercept": bmb.Prior("Normal", mu=0, sigma=1),
    "1|study": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=1))
}
model_hierarchical = bmb.Model("p(reuse, total) ~ 1 + group + (1|study)", combined_data, family="binomial", priors=priors)
model_hierarchical

       Formula: p(reuse, total) ~ 1 + group + (1|study)
        Family: binomial
          Link: p = logit
  Observations: 14
        Priors: 
    target = p
        Common-level effects
            Intercept ~ Normal(mu: 0.0, sigma: 1.0)
            group ~ Normal(mu: 0.0, sigma: 5.0)
        
        Group-level effects
            1|study ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 1.0))

# 5. Fitting Model

In [42]:
idata_hierarchical = model_hierarchical.fit(random_seed=42)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, group, 1|study_sigma, 1|study_offset]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 13 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


In [45]:
print(idata_hierarchical.posterior)


<xarray.Dataset> Size: 328kB
Dimensions:            (chain: 4, draw: 1000, group_dim: 1, study__factor_dim: 7)
Coordinates:
  * chain              (chain) int64 32B 0 1 2 3
  * draw               (draw) int64 8kB 0 1 2 3 4 5 ... 994 995 996 997 998 999
  * group_dim          (group_dim) <U6 24B 'social'
  * study__factor_dim  (study__factor_dim) <U1 28B '1' '2' '3' '4' '5' '6' '7'
Data variables:
    Intercept          (chain, draw) float64 32kB 0.1683 0.1897 ... 0.05734
    group              (chain, draw, group_dim) float64 32kB 0.08793 ... 0.04418
    1|study_sigma      (chain, draw) float64 32kB 1.052 1.137 ... 1.254 1.257
    1|study            (chain, draw, study__factor_dim) float64 224kB -0.5754...
Attributes:
    created_at:                  2025-09-22T18:03:59.211093
    arviz_version:               0.17.1
    inference_library:           pymc
    inference_library_version:   5.12.0
    sampling_time:               13.24306583404541
    tuning_steps:                1000
    m

In [46]:
summ = az.summary(
    idata_hierarchical,
    var_names=["Intercept", "group", "1|study_sigma"],
    hdi_prob=0.90
)
print(summ)

                mean     sd  hdi_5%  hdi_95%  mcse_mean  mcse_sd  ess_bulk  \
Intercept      0.381  0.355  -0.215    0.956      0.014    0.010     687.0   
group[social]  0.208  0.075   0.087    0.336      0.001    0.001    2711.0   
1|study_sigma  0.984  0.300   0.542    1.448      0.011    0.008     836.0   

               ess_tail  r_hat  
Intercept         667.0    1.0  
group[social]    2371.0    1.0  
1|study_sigma     764.0    1.0  


**Research Question**  
Does the social norm intervention increase towel reuse compared to the control condition?

**Hypotheses**  

- **Null hypothesis (H₀):**  
  The intervention has no effect.  
  Mathematically: `group[social] = 0`.

- **Alternative hypothesis (H₁):**  
  The intervention is effective. The social norm group shows a higher reuse rate than the control group.  
  Mathematically: `group[social] > 0`.

### Interpretation

- The posterior probability that the intervention effect is greater than zero is nearly 1.0, 
  indicating very strong evidence for effectiveness.
- The posterior mean of `group[social]` is about 0.21 (log-odds), with a 90% HDI of [0.09, 0.34].
- This corresponds to approximately a 5–6 percentage point increase in towel reuse under the social norm condition compared to control.
